https://euske.github.io/introdl/lec6/index.html

GPU は CPU に比べて単純な処理しかできないが、CPU に比べて はるかに多くの処理を並列実行できるため、ある種のアルゴリズムに対しては CPU に比べて数倍〜数十倍の速度が出せる。ニューラルネットワークで 行われる演算はほとんどが足し算と掛け算なので、 とくに GPU で処理するのに向いているといえる.  
NVIDIA が提供する CUDA 開発キットには GPU 用のコードが 生成できるよう拡張された C/C++コンパイラ (nvcc) が付属している。

In [8]:
import torch

torch.cuda.is_available()

True

機械学習における「テンソル」は「多次元配列」とほぼ同義である。 したがって Tensor型の基本機能も NumPy における ndarray型とほとんど同じであり、使い方もわざと ndarray型に似せてある。 ただし、以下のような機能が追加されている:

Tensor上のデータは、CPU上の主記憶か、 あるいは GPU上のメモリのどちらに格納するか選ぶことができる。
Tensor上の各数値は、 それが計算されたときの勾配 (grad) を保持することができる。


In [9]:
x1 = torch.tensor([1,2,3])  # x1はCPU上に作成される。

In [10]:
x1

tensor([1, 2, 3])

In [11]:
x2 = x1.to('cuda')          # x1をGPUに転送し、x2とする。

In [12]:
x2

tensor([1, 2, 3], device='cuda:0')

In [13]:
x3 = x2.to('cpu')           # x2をCPUに転送し、x3とする。

In [14]:
x3

tensor([1, 2, 3])

In [15]:
x2*x2

tensor([1, 4, 9], device='cuda:0')

In [16]:
x1*x2      # CPU上とGPU上にあるデータは互いに計算できない。

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu!

## 演習6-5. CUDA を使ってテンソルを計算する (CUDA が使用可能な環境のみ)

In [17]:
import time
# timeit: テンソルを二乗し、計算にかかった時間を表示する。
def timeit(x):
    t0 = time.time()
    x = torch.mm(x, x)
    dt = time.time() - t0
    print(dt)
    return dt

CPU

In [19]:
# 100×100のランダムな行列を二乗する。
x = torch.rand(100,100)
timeit(x)
# 10000×10000のランダムな行列を二乗する。
x = torch.rand(10000,10000)
timeit(x)

0.00898122787475586
1.2357871532440186


1.2357871532440186

In [20]:
# 100×100のランダムな行列を二乗する。
x = torch.rand(100,100).to('cuda')
timeit(x)
# 10000×10000のランダムな行列を二乗する。
x = torch.rand(10000,10000).to('cuda')
timeit(x)

0.45891499519348145
0.0008440017700195312


0.0008440017700195312

## 3. PyTorch を使った MNIST の実装

 PyTorch では、計算に使うテンソルが GPU 上にあれば GPU 上で計算が行われる。 そのため、以上のコードを GPU に対応させるのは容易である。

In [21]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

# MNISTを処理するニューラルネットワーク。
class MNISTNet(nn.Module):

    # 各レイヤーの初期化。
    def __init__(self):
        nn.Module.__init__(self)
        # 畳み込み: 入力1チャンネル、出力10チャンネル、カーネル3×3。
        self.conv1 = nn.Conv2d(1, 10, 3)
        # Max Pooling: 1/2に縮める。
        self.pool1 = nn.MaxPool2d(2)
        # 畳み込み: 入力10チャンネル、出力20チャンネル、カーネル3×3。
        self.conv2 = nn.Conv2d(10, 20, 3)
        # Max Pooling: 1/2に縮める。
        self.pool2 = nn.MaxPool2d(2)
        # 全接続 (fully connected): 入力500ノード、出力10ノード。
        self.fc1 = nn.Linear(20*5*5, 10)
        return

    # 与えらえたミニバッチ x を処理する。
    def forward(self, x):
        # x: (N × 1 × 28 × 28)
        x = self.conv1(x)
        x = F.relu(x)
        # x: (N × 10 × 26 × 26)
        x = self.pool1(x)
        # x: (N × 10 × 13 × 13)
        x = self.conv2(x)
        x = F.relu(x)
        # x: (N × 20 × 11 × 11)
        x = self.pool2(x)
        # x: (N × 20 × 5 × 5)
        x = x.reshape(len(x), 20*5*5)
        # x: (N × 500)
        x = self.fc1(x)
        # x: (N × 10)
        return x

# 実際のインスタンスを作成。
model = MNISTNet().to('cuda')  # ニューラルネットワークの重み・バイアスを GPU に転送する

In [22]:
print(model)

MNISTNet(
  (conv1): Conv2d(1, 10, kernel_size=(3, 3), stride=(1, 1))
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(10, 20, kernel_size=(3, 3), stride=(1, 1))
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=500, out_features=10, bias=True)
)


In [29]:
# ミニバッチごとの訓練データを用意する。
train_images = load_mnist('/raw/train-images-idx3-ubyte.gz') 
train_labels = load_mnist('/raw/train-labels-idx1-ubyte.gz') 
# ニューラルネットワークを訓練モードにする。
model.train()
# 最適化器と学習率を定義する。
optimizer = optim.SGD(model.parameters(), lr=0.01)
n = 0
# 各ミニバッチを処理する。
for (images,labels) in zip(train_images, train_labels):
    images = images.reshape(len(images), 1, 28, 28)
    # 入力をfloat型のテンソルに変換。
    inputs = torch.tensor(images).float()
    # 正解をlong型のテンソルに変換。
    targets = torch.tensor(labels).long()
    # すべての勾配(.grad)をクリアしておく。
    optimizer.zero_grad()
    # 与えられたミニバッチをニューラルネットワークに処理させる。
    output = model(inputs)
    # 損失を計算する。
    loss = F.cross_entropy(output, labels)
    # 勾配を計算する。
    loss.backward()
    # 重み・バイアスを更新する。
    optimizer.step()
    n += len(images)
    print(n, loss.item())

NameError: name 'load_mnist' is not defined